Checking that missing dates are actually missing like we think

In [1]:
from utils_filter import *
from utils_datetime import *
from utils_geography import *
from utils_plotting import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors

Checking in grid outlooks

In [ ]:
missing_dates = ['200204190000', '200204200000', '200204210000', '200204250000', '200205060000', '200205250000', '200207310000', '200208130000', '200208300000', '200211090000', '200212230000', '200302030000', '200303250000', '200304140000', '200304150000', '200304160000', '200305100000', '200306250000', '200306280000', '200307270000', '200307280000', '200309030000', '200312280000', '200404020000', '200404140000', '200405230000', '200408090000', '200410140000', '200503300000', '200506060000', '200508030000', '200701040000', '200905280000', '201105210000', '202005240000', '202106130000']

In [2]:
grid_outlook_location = 'data/outlooks/grid_outlooks.nc'
grid_outlooks = xr.open_dataset(grid_outlook_location)

In [3]:
max_outlooks = grid_outlooks.max(dim = ['x', 'y'])['prob']

dates = max_outlooks['time']
thresholds = {'': .15,
              ' Tornado': .05,
              ' Wind': .15,
              ' Hail': .15}
days = ['Day 3', 'Day 2 7', 'Day 2 17', 'Day 1']
hazard_days = ['Day 2 7', 'Day 2 17', 'Day 1']
investigate_dates = []
for date in dates:
    max_outlooks_date = max_outlooks.sel(time = date)
    for hazard in ['', ' Tornado', ' Wind', ' Hail']:
        if hazard == '':
            this_days = days
        else:
            this_days = hazard_days
        outlook_keys = [item + hazard for item in this_days]
        max_prob = max_outlooks_date.sel(outlook = outlook_keys).max()
        if max_prob >= thresholds[hazard]:
            for outlook_key in outlook_keys:
                if max_outlooks_date.sel(outlook = outlook_key) == 0 and outlook_key == 'Day 1':
                    print(str(date.data) + ' ' + str(outlook_key))
                    investigate_dates.append(str(date.data))

                

print(investigate_dates)

200204250000 Day 1
200208300000 Day 1
200304150000 Day 1
200304160000 Day 1
200306250000 Day 1
200307270000 Day 1
200307280000 Day 1
200312280000 Day 1
200404140000 Day 1
200408090000 Day 1
200905280000 Day 1
201105210000 Day 1
202005240000 Day 1
['200204250000', '200208300000', '200304150000', '200304160000', '200306250000', '200307270000', '200307280000', '200312280000', '200404140000', '200408090000', '200905280000', '201105210000', '202005240000']


In [ ]:
print(missing_dates)
print(investigate_dates)
len(investigate_dates)

In [ ]:
pph = xr.open_dataset('data/pph/labelled_pph.nc')
investigate_pph = pph.sel(time = investigate_dates)

In [ ]:
mdt_investigate_dates = investigate_pph['MAX_CAT'].isin(['MDT', 'HIGH'])
mdt_investigate_dates
mdt_investigate_pph = investigate_pph.sel(time = mdt_investigate_dates)
mdt_investigate_pph

Checking in raw data

In [4]:
raw_dir = '~/Downloads'
outlook_raw_location = raw_dir + '/outlooks'

# Read convective outlooks into outlooks
year_list = [[1987, 1991], [1992, 1999], [2000, 2007], [2008, 2015], [2016, 2023]]
outlook_list = []

for years, i in zip(year_list, range(len(year_list))):
    print('reading file ' + str(i) + ', years ' + str(years[0]) + '-' + str(years[1]))
    gdf = gp.read_file(outlook_raw_location + '/outlooks_' + str(years[0]) + '01010000_' + str(years[1]) + '12312359')
    outlook_list.append(gdf)

# Concatenate all GeoDataFrames into one
outlooks_original = pd.concat(outlook_list, ignore_index=True)

print('files read')
    
outlooks_original

reading file 0, years 1987-1991
reading file 1, years 1992-1999
reading file 2, years 2000-2007
reading file 3, years 2008-2015
reading file 4, years 2016-2023
files read


,ISSUE,EXPIRE,PRODISS,TYPE,DAY,THRESHOLD,CATEGORY,CYCLE,geometry
0,198701011200,198701021200,198701010635,C,1,TSTM,CATEGORICAL,-1,"MULTIPOLYGON (((-80.69500 29.36500, -80.68200 ..."
1,198701011500,198701021200,198701011441,C,1,TSTM,CATEGORICAL,-1,"POLYGON ((-70.15100 42.90900, -70.14600 42.899..."
2,198701011900,198701021200,198701011849,C,1,TSTM,CATEGORICAL,-1,"POLYGON ((-73.73200 39.90700, -73.73200 39.892..."
3,198701021200,198701031200,198701020637,C,1,TSTM,CATEGORICAL,-1,"POLYGON ((-124.43300 41.91800, -124.44100 41.9..."
4,198701031200,198701041200,198701020725,C,2,None,None,-1,None
...,...,...,...,...,...,...,...,...,...
366366,202401021200,202401031200,202312310815,C,3,TSTM,CATEGORICAL,8,"POLYGON ((-90.74500 28.71100, -91.08300 28.732..."
366367,202312311300,202401011200,202312311236,C,1,None,None,13,None
366368,202312311630,202401011200,202312311558,C,1,None,None,16,None
366369,202401011200,202401021200,202312311712,C,2,None,None,17,None


In [39]:
for date in investigate_dates:
    print(date)
    print(outlooks_original[(outlooks_original['ISSUE'] == date[0:8] + '1200') & (outlooks_original['DAY'] == 1)]['CYCLE'].unique())


200204250000
[6]
200208300000
[]
200304150000
[]
200304160000
[]
200306250000
[6]
200307270000
[]
200307280000
[]
200312280000
[6]
200404140000
[6]
200408090000
[]
200905280000
[]
201105210000
[]
202005240000
[-1  6]


In [ ]:
outlooks_original[(outlooks_original['ISSUE'] == '202005241200')& (outlooks_original['DAY'] == 1)]

,ISSUE,EXPIRE,PRODISS,TYPE,DAY,THRESHOLD,CATEGORY,CYCLE,geometry
310603,202005241200,202005251200,202005240549,C,1,None,None,-1,None
310609,202005241200,202005251200,202005240653,C,1,None,None,6,None


: 

Look for dates with day 2 17 but no day 1 6 outlooks--there are a lot more, but with some spot checking they generally seem to have actually happened like this

In [12]:
data_location = 'data'
# outlooks, pph, reports = read_datasets(data_location)
outlook1 = gp.read_file(f"{data_location}/outlooks/all_outlooks_1.shp", engine="pyogrio")
outlook2 = gp.read_file(f"{data_location}/outlooks/all_outlooks_2.shp", engine="pyogrio")
outlooks = gp.GeoDataFrame(pd.concat([outlook1, outlook2], ignore_index=True), crs=outlook1.crs)

In [34]:
# Pre-filter relevant cycles to speed things up
outlook_subset = outlooks[(outlooks['CYCLE'].isin([6, 17])) & outlooks['CATEGORY'].isin(['CATEGORICAL', 'WIND', 'HAIL', 'TORNADO'])]

# Create a pivot table with counts of each cycle per (CATEGORY, DATE)
pivot = outlook_subset.pivot_table(index=['DATE'], 
                                   columns='CYCLE', 
                                   aggfunc='size', 
                                   fill_value=0)



# Check where 17 exists and 6 does not
condition =  (pivot.get(17, 0) > 0) & (pivot.get(6, 0) == 0)

for date in pivot.index[condition]:
    print(date)

len(pivot.index[condition])

199504240000
199507190000
199603290000
199607010000
199607170000
199709040000
199710280000
199712150000
199712190000
199803030000
199810260000
199812090000
199903290000
199910060000
199911030000
199911080000
199911180000
199912050000
200001070000
200001110000
200002020000
200008150000
200009180000
200009230000
200011110000
200101210000
200102170000
200103050000
200111100000
200112060000
200112290000
200201180000
200201190000
200202040000
200202090000
200202240000
200203260000
200211170000
200301070000
200301080000
200301160000
200301200000
200302050000
200302090000
200303030000
200303110000
200304150000
200304160000
200307270000
200307280000
200310180000
200312010000
200401060000
200401090000
200402100000
200408090000
200411060000
200412110000
200501210000
200512070000
200602070000
200602180000
200611250000
200612080000
200701200000
200712060000
200802020000
200804130000
200812130000
200905280000
200910310000
200911030000
201002110000
201002160000
201011260000
201101030000
201102080000

119

In [ ]:
['200204250000', '200208300000', '200304150000', '200304160000', '200306250000', '200307270000', '200307280000', '200312280000', '200404140000', '200408090000', '200905280000', '201105210000', '202005240000']

,ISSUE,EXPIRE,PRODISS,TYPE,DAY,THRESHOLD,CATEGORY,CYCLE,DATE,geometry
0,198701011200,198701021200,198701010635,C,1,TSTM,CATEGORICAL,6,198701010000,"MULTIPOLYGON (((-80.69500 29.36500, -80.68200 ..."
1,198701011500,198701021200,198701011441,C,1,TSTM,CATEGORICAL,-1,198701010000,"POLYGON ((-70.15100 42.90900, -70.14600 42.899..."
2,198701011900,198701021200,198701011849,C,1,TSTM,CATEGORICAL,-1,198701010000,"POLYGON ((-73.73200 39.90700, -73.73200 39.892..."
3,198701021200,198701031200,198701020637,C,1,TSTM,CATEGORICAL,6,198701020000,"POLYGON ((-124.43300 41.91800, -124.44100 41.9..."
4,198701031200,198701041200,198701020725,C,2,None,None,7,198701030000,None
...,...,...,...,...,...,...,...,...,...,...
366365,202401021200,202401031200,202312310815,C,3,TSTM,CATEGORICAL,8,202401020000,"POLYGON ((-90.74500 28.71100, -91.08300 28.732..."
366366,202312311300,202401011200,202312311236,C,1,None,None,13,202312310000,None
366367,202312311630,202401011200,202312311558,C,1,None,None,16,202312310000,None
366368,202401011200,202401021200,202312311712,C,2,None,None,17,202401010000,None
